In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
import collections
import torch
import torch.nn as nn
import torch.nn.functional as func
import torch.autograd as autograd
import random
import pandas as pd
import datetime

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Device name:',device)

Device name: cuda:0


In [2]:

eps = torch.finfo(torch.float32).eps

class PINN(nn.Module):
    def __init__(self, input_num, output_num, receivers, senders, radius, bias=True):
        super().__init__()
        
        self.bias=bias
        self.radius_tg = radius
        self.fc1 = nn.Linear(input_num,32, bias=bias)
        self.fc2 = nn.Linear(32,32, bias=bias)
        self.fc3 = nn.Linear(32,32, bias=bias)
        self.fc4 = nn.Linear(32, output_num, bias=bias)
        self.reset_parameters()
        
        self.receivers=receivers
        self.senders=senders
        
        
        self.lambda1_v = torch.FloatTensor(1).uniform_(-0.1, 0.1).requires_grad_(True).to(device)
        self.lambda2_v = torch.FloatTensor(1).uniform_(-0.1, 0.1).requires_grad_(True).to(device)
        self.lambda3_v = torch.FloatTensor(1).uniform_(-0.1, 0.1).requires_grad_(True).to(device)
        self.lambda4_v = torch.FloatTensor(1).uniform_(-0.1, 0.1).requires_grad_(True).to(device)
        self.power1_v = torch.FloatTensor(1).uniform_(-0.1, 0.1).requires_grad_(True).to(device) 
        self.power2_v = torch.FloatTensor(1).uniform_(-0.1, 0.1).requires_grad_(True).to(device)
        self.power3_v = torch.FloatTensor(1).uniform_(-0.1, 0.1).requires_grad_(True).to(device)
        
        self.lambda1_v = nn.Parameter(self.lambda1_v)
        self.lambda2_v = nn.Parameter(self.lambda2_v)
        self.lambda3_v = nn.Parameter(self.lambda3_v)
        self.lambda4_v = nn.Parameter(self.lambda4_v)
        self.power1_v = nn.Parameter(self.power1_v)
        self.power2_v = nn.Parameter(self.power2_v)
        self.power3_v = nn.Parameter(self.power3_v)

        
    def forward(self, x):
        self.input = x
        self.row_num = x.size(0)
        x_max = x[-1].item()/2
        
        x=(self.input-x_max)/x_max
        self.x1=torch.tanh(self.fc1(x))
        self.x2=torch.tanh(self.fc2(self.x1))
        self.x3=torch.tanh(self.fc3(self.x2))
        self.x4 = self.fc4(self.x3)
        output =radius*self.x4
        
        return output

    def reset_parameters(self) -> None:
        nn.init.xavier_uniform_(self.fc1.weight, gain = nn.init.calculate_gain('tanh'))
        nn.init.xavier_uniform_(self.fc2.weight, gain = nn.init.calculate_gain('tanh'))
        nn.init.xavier_uniform_(self.fc3.weight, gain = nn.init.calculate_gain('tanh'))
        nn.init.xavier_uniform_(self.fc4.weight, gain=1)
        
        if self.bias:
            nn.init.constant_(self.fc1.bias, 0.1)
            nn.init.constant_(self.fc2.bias, 0.1)
            nn.init.constant_(self.fc3.bias, 0.1)
            nn.init.constant_(self.fc4.bias, 0.1)
            
    def loss_func(self, pred, initial, L=5, v0=2, order_para_tg=1, int_range=3, tau=10):
        loss_mse = nn.MSELoss().to(device)
        
        #calculate gradients
        
        half=pred.size(1)//2
        center_x = torch.mean(pred[:,0:half], dim=1, keepdim=True)
        center_y = torch.mean(pred[:,half:], dim=1, keepdim=True)
        
        for i in range(pred.size(1)):
            temp=torch.zeros_like(pred)
            temp[:,i]=1
            grads, = autograd.grad(pred, self.input, grad_outputs=temp, create_graph=True)
            if i==0:
                self.u = grads
            else:
                self.u = torch.hstack((self.u, grads))

        self.u_mag=torch.sqrt(torch.square(self.u[:,0:half])+torch.square(self.u[:,half:]))

        for i in range(pred.size(1)):
            temp=torch.zeros_like(pred)
            temp[:,i]=1
            grads, = autograd.grad(self.u, self.input, grad_outputs=temp, create_graph=True)
            if i==0:
                self.accel = grads
            else:
                self.accel = torch.hstack((self.accel, grads))
        
        N_time=pred.size(0)
        
        
        ####### calculate distance with periodic boundary condition #########
        temp_x = pred[:,0:half] - 2*L*torch.div(pred[:,0:half],2*L,rounding_mode='trunc')
        temp_y = pred[:,half:] - 2*L*torch.div(pred[:,half:],2*L,rounding_mode='trunc')
        pos_x = temp_x - 2*L*torch.div(temp_x,L,rounding_mode='trunc')
        pos_y = temp_y - 2*L*torch.div(temp_y,L,rounding_mode='trunc')
        pos_x_r = pos_x[:, self.receivers.flatten().type(torch.int64)]
        pos_y_r = pos_y[:, self.receivers.flatten().type(torch.int64)]
        pos_x_s = pos_x[:, self.senders.flatten().type(torch.int64)]
        pos_y_s = pos_y[:, self.senders.flatten().type(torch.int64)]

        dist_x = pos_x_r - pos_x_s
        dist_y = pos_y_r - pos_y_s
        dist=torch.sqrt(torch.square(dist_x)+torch.square(dist_y))
        dist_value = dist.detach()
        r_mag = torch.sqrt(torch.square(pos_x-center_x) + torch.square(pos_y-center_x))
        
        ##### calculate relative velocity distance #########
        vel_x = self.u[:,0:half]
        vel_y = self.u[:,half:]
        vel_x_r = vel_x[:, self.receivers.flatten().type(torch.int64)]
        vel_y_r = vel_y[:, self.receivers.flatten().type(torch.int64)]
        vel_x_s = vel_x[:, self.senders.flatten().type(torch.int64)]
        vel_y_s = vel_y[:, self.senders.flatten().type(torch.int64)]
        
        vel_rel_x = vel_x_r - vel_x_s
        vel_rel_y = vel_y_r - vel_y_s
        vel_rel = torch.sqrt(torch.square(vel_rel_x)+torch.square(vel_rel_y))
        
        ######## calculate loss between output edges and f_inter matgnitude from physics ######
        local_dist_func = torch.heaviside(int_range - dist_value, torch.tensor([1.], device = device))
        
        self.f_inter_x = (vel_rel_x/vel_rel)*(self.lambda1_v*(dist**self.power1_v) + self.lambda2_v*(dist**self.power2_v)+\
                                            self.lambda3_v*(dist**self.power3_v) + self.lambda4_v)*local_dist_func
        
        self.f_inter_y = (vel_rel_y/vel_rel)*(self.lambda1_v*(dist**self.power1_v) + self.lambda2_v*(dist**self.power2_v)+\
                                            self.lambda3_v*(dist**self.power3_v) + self.lambda4_v)*local_dist_func
        


        f_agg_x = torch.zeros(self.row_num, half).to(device)
        f_agg_y = torch.zeros(self.row_num, half).to(device)

        f_agg_x = f_agg_x.scatter_add_(1, self.receivers.flatten().repeat(self.row_num,1).type(torch.int64), self.f_inter_x).to(device)
        f_agg_y = f_agg_y.scatter_add_(1, self.receivers.flatten().repeat(self.row_num,1).type(torch.int64), self.f_inter_y).to(device)
        
        ######## Self propelled term #########        
        
        sp_x = 1*self.u[:,0:half]*(v0/self.u_mag-1)
        sp_y = 1*self.u[:,half:]*(v0/self.u_mag-1)
        
        ode_x = self.accel[:,0:half]-sp_x-f_agg_x
        ode_y = self.accel[:,half:]-sp_y-f_agg_y

        self.loss_force = torch.sum(torch.square(ode_x)+torch.square(ode_y))/(pred.size(0)*half)

        
        
        ########### Loss from initial condition ##########
        self.loss_pos = torch.mean(torch.square(pred[0,:]-torch.flatten(initial[0:2,:],start_dim=0)))
        self.loss_vel = torch.mean(torch.square(self.u[0,:]-torch.flatten(initial[2:4,:],start_dim=0)))
        
        order_para_init = torch.sqrt(torch.square(torch.sum(initial[2,:]))+torch.square(torch.sum(initial[3,:])))/\
        torch.sum(torch.sqrt(torch.square(initial[2,:])+torch.square(initial[3,:])))
        
        ######## Order parameter #########
        self.data_pt = torch.arange(0, N_time, N_time//(N_time//10)).to(device)
        
        denom = torch.sum(self.u_mag, dim=1)
        lin_mom = torch.sqrt(torch.sum(self.u[:,0:half],dim=1)**2 + torch.sum(self.u[:,half:],dim=1)**2)
        order_para =  lin_mom/(half*v0)
        
        self.loss_ord = torch.mean(torch.square(order_para[self.data_pt] \
                                                - ((order_para_tg-order_para_init)/N_time*self.data_pt + order_para_init)))
        self.loss_ord_f = torch.mean(torch.square(order_para[-1] - order_para_tg))


        loss = 5*self.loss_pos + 5*self.loss_vel+ 5*self.loss_ord_f + 5*self.loss_ord + 1*self.loss_force

 
        return loss


In [3]:
##########Inital condition ##############

position_list_x=[]
position_list_y=[]
velocity_list_x=[]
velocity_list_y=[]

### if you want to fix random seed, use it ###
# random.seed(101) 
int_range = 2
field = 40**(0.5)/4
for i in range(40):
    position_list_x.append(random.uniform(-field, field))
    position_list_y.append(random.uniform(-field, field))
    velocity_list_x.append(random.uniform(-2,2))
    velocity_list_y.append(random.uniform(-2,2))

eps = torch.finfo(torch.float32).eps

initial = torch.tensor([position_list_x, position_list_y, velocity_list_x, velocity_list_y], dtype=torch.float32).to(device)

senders=[]
receivers=[]

for i in range(initial.size(dim=1)):
    for j in range(initial.size(dim=1)-1):
        receivers.append([i])
    for k in range(initial.size(dim=1)):
        if k!=i:
            senders.append([k])


senders_G= torch.tensor(senders, dtype=torch.float32).to(device)  #index of the sender node for edge
receivers_G= torch.tensor(receivers, dtype=torch.float32).to(device)  #index of the receiver node for edge


In [4]:
def closure():
    model.train()
    optimizer_LBFGS.zero_grad()
    pred=model(t)
    loss=model.loss_func(pred, initial, L =field, v0=v0, order_para_tg=1, int_range=int_range, tau=time_end-dt)

    if torch.isfinite(loss).item:
        loss.backward() 
    else:
        pass

    return loss

In [5]:

#### define hyperparameter####

dt=0.1
time_end = 10+dt
v0=2
epoch_adam=200
epoch_LBFGS=1000
patience=-21
tolerance=1e-4

##define input
t=torch.arange(0, time_end-eps, dt, dtype=torch.float32)
t = t.reshape(len(t),1).to(device)
t.requires_grad=True
out_num = initial.size(1)*2

radius = field
int_range = 2


retry_num = 100

print("Training Started")
for ii in range(retry_num):
    loss_value=[]
    
    ##define model
    model=PINN(input_num=1, output_num=out_num, receivers=receivers_G, senders=senders_G, radius = radius).to(device)


    ## ADAM ##
    for i in range(epoch_adam):
        learning_rate = 0.001
        optimizer_adam=torch.optim.Adam(model.parameters(), lr=learning_rate)

        model.train()
        optimizer_adam.zero_grad()

        pred=model(t)
        loss=model.loss_func(pred, initial, L =field, v0=v0, order_para_tg=1, int_range=int_range, tau=time_end-dt)

        loss.backward()
        optimizer_adam.step()
        loss_value.append(loss.item())
        
        ### Early Stop condition ###
        if i>21:
            if loss_value[-1] <= loss_value[-2] and abs(loss_value[-1]-np.mean(loss_value[patience:-1])) < tolerance:
                break



    ##LBFGS ##
    optimizer_LBFGS=torch.optim.LBFGS(model.parameters(), lr=0.1, max_iter=20, line_search_fn = 'strong_wolfe')

    for i in range(epoch_LBFGS):

        loss_prev = optimizer_LBFGS.step(closure)
        if torch.isfinite(loss_prev) == False:
            break
            
        loss_value.append(loss_prev.item())
        ### Early Stop condition ###
        if i>21:
            if loss_value[-1] <= loss_value[-2] and abs(loss_value[-1]-np.mean(loss_value[patience:-1])) < tolerance:
                break
    
    ### Stopping Condition ####
    if model.loss_ord_f.item()<0.1 and model.loss_force.item()<0.3:
        break

print("Training Stopped") 
print("lambda1_v = %.3e \nlambda2_v = %.3e \nlambda3_v = %.3e \nlambda4_v = %.3e" \
      %(model.lambda1_v, model.lambda2_v, model.lambda3_v, model.lambda4_v))
print("power1_v = %.3e \npower2_v = %.3e \npower3_v = %.3e" \
      %(model.power1_v, model.power2_v, model.power3_v))
print("int_range = %.3e" %(int_range))

Training Started
Training Stopped
lambda1_v = -8.414e-03 
lambda2_v = 1.833e-01 
lambda3_v = -4.845e-03 
lambda4_v = -2.210e-01
power1_v = -9.465e-03 
power2_v = -3.977e-02 
power3_v = 3.053e-01
int_range = 1.000e+00
